# **GPU設定**
まず左上の ***ランタイム > ランタイムのタイプを変更*** からハードウェアアクセラレータをGPUに変更する。


# **自分のGoogle Driveにマウント**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Tensorflow Object Detection APIの設定**

### Protobufのインストール・コンパイル
v3.20.0以降のProtobufだとエラーがでるため、v3.19.4をインストール

In [ ]:
!curl -OL https://github.com/protocolbuffers/protobuf/releases/download/v3.19.4/protoc-3.19.4-linux-x86_64.zip
!unzip protoc-3.19.4-linux-x86_64.zip -d protoc3
!sudo mv protoc3/bin/* /usr/local/bin/
!sudo mv protoc3/include/* /usr/local/include/
!rm -rf protoc3 protoc-3.19.4-linux-x86_64.zip

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models
%cd /content/models/research

!/usr/local/bin/protoc object_detection/protos/*.proto --python_out=.

### Object Detection APIに必要なパッケージのインストール
以下でsetup.pyを動かすとpipの依存関係による赤文字のエラーがでるが、インストールは成功するので気にしなくて良い。

In [ ]:
!cp /content/models/research/object_detection/packages/tf2/setup.py .
!python -m pip install . --use-feature=in-tree-build

In [ ]:
# インストールが上手くいったか確認
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

上記のsetup.pyで最新のTensorflow(2022年7月現在：2.9.1)がインストールされてしまう。
<br>そのまま学習するとエラーが出るので、以下で必要なものを適宜インストール・アップグレードする。

<br>・aptでインストールできるCuda, cuDNNの最新版(2022年7月現在：libcudnn8_8.4.1.50-1+cuda11.6_amd64.deb)

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

・opencv-pythonのアップグレード

In [ ]:
!pip install -U opencv-python

#**学習**
5000くらい回すと良い感じの精度になったが、最初は1000くらいで回してlossの値を見て調節していくと良いかもしれない。

In [ ]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path="/content/drive/MyDrive/Tensorflow_ODA_DriveDir/pipeline.config" \
    --model_dir="/content/drive/MyDrive/Tensorflow_ODA_DriveDir/trained" \
    --num_train_steps=5000 \
    --alsologtostderr \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=100

# **Saved_Model形式にエクスポート**
Tensorflow_ODA_DriveDir内にoutputフォルダを作り、最終的にその中にできたsaved_modelをダウンロードして使う。

In [ ]:
!python object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path="/content/drive/MyDrive/Tensorflow_ODA_DriveDir/pipeline.config" \
    --trained_checkpoint_dir="/content/drive/MyDrive/Tensorflow_ODA_DriveDir/trained" \
    --output_directory="/content/drive/MyDrive/Tensorflow_ODA_DriveDir/output"